# Курс "Машиное обучение в бизнесе"¶

## Практическое задание урока 3. Связь бизнес-показателей и DS-метрик

### Задания 1 - 4

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других

#### Решение

In [16]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [17]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [18]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

К полям:
- gender, cholesterol применим OHE-кодирование
- age, height, weight, ap_hi, ap_lo - standardScaler
- gluc, smoke, alco, active - оставим пока как есть

In [19]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Теперь объединим все наши трансформеры с помощью FeatureUnion

In [20]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb, lightgbm as lgbm, catboost as catb
from catboost import CatBoostClassifier

from sklearn.metrics import f1_score, roc_auc_score, precision_score, \
                            classification_report, precision_recall_curve, confusion_matrix

In [22]:
# Функция обучения модели и расчета метрик
def get_metrics(classifier):
    from sklearn.metrics import f1_score
    from sklearn.metrics import roc_auc_score
    
    #запустим кросс-валидацию
    cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)

    #обучим пайплайн на всем тренировочном датасете
    classifier.fit(X_train, y_train)
    y_score = classifier.predict_proba(X_test)[:, 1]
    
    # расчтаем Precision, Recall, F_score, Roc auc
    b = 1
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1 + b**2) * (precision * recall) / (b**2 * precision + recall)
    roc_auc = roc_auc_score(y_test, y_score)
    ix = np.argmax(fscore)
    
 
    return roc_auc, precision[ix], recall[ix], fscore[ix]

In [23]:
classifier_1 = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])

classifier_2 = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

classifier_3 = Pipeline([
    ('features',feats),
    ('classifier', DecisionTreeClassifier(max_depth=4, random_state=42)),
])

frozen_params = {
    'eval_metric': 'F1',
    'auto_class_weights': 'Balanced',
    'silent': True,
    'one_hot_max_size': 20,
    'early_stopping_rounds': 50,
    'boosting_type': 'Ordered',
    'allow_writing_files': False
}

classifier_4 = Pipeline([
    ('features',feats),
    ('classifier', CatBoostClassifier(
    **frozen_params,
    depth=4,
    iterations=400,
    learning_rate=0.1,
    l2_leaf_reg=2.5,
    bagging_temperature=1.5
)),
])

In [24]:
metrics = {'LogisticRegression': classifier_1, 
           'RandomForestClassifier': classifier_2, 
           'DecisionTreeClassifier': classifier_3,
          'CatBoostClassifier': classifier_4
          }
info = pd.DataFrame(columns=metrics.keys())

In [25]:
%%time

for metric, value in metrics.items():
    print(metric, value)
    roc_auc, precision, recall, fscore = get_metrics(value)
    info.loc[f'roc_auc', metric] = roc_auc
    info.loc[f'precision', metric] = precision
    info.loc[f'recall', metric] = recall
    info.loc[f'fscore', metric] = fscore

LogisticRegression Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('age',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  NumberSelector(key='age')),
                                                                 ('standard',
                                                                  StandardScaler(copy=True,
                                                                                 with_mean=True,
                                                                                 with_std=True))],
                                                          verbose=False)),
                                                ('height',
                                                 Pipeline(memory=None,
                 

Wall time: 7min 13s


In [26]:
info

,LogisticRegression,RandomForestClassifier,DecisionTreeClassifier,CatBoostClassifier
roc_auc,0.784035,0.771037,0.788519,0.799925
precision,0.647431,0.642669,0.646918,0.698779
recall,0.837558,0.815553,0.840323,0.784677
fscore,0.730323,0.718863,0.731045,0.739241


**Вывод**

Из примененных моделей с задачей лучше справился CatBoost, в котором получены наибольшие значения ROC_AUC и F-меры.

### Задание 5 (опционально)

5. (опциональный вопрос) какой график (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого, например, 1 к 1000).

p.s. В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно).
Допустим, у нас две модели:

первая помечает 100 объектов как класс 1, но TP = 90
вторая помечает 1000 объектов как класс 1, но TP такой же - 90
Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

#### Решение

***Первая модель*** возвращает 100 документов, 90 из которых релевантны

In [52]:
TP=90
FN=10
FP=10
TN=99890
TPR = TP/(TP+FN)
FPR = FP/(FP+TN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)

print(f'TPR = {TPR:.4f}, FPR = {FPR:.4f}, precision = {precision:.4f}, recall = {recall:.4f}')

TPR = 0.9000, FPR = 0.0001, precision = 0.9000, recall = 0.9000


***Вторая модель*** возвращает 1000 документов, 90 из которых релевантны

In [53]:
TP=90
FN=10
FP=910
TN=97990
TPR = TP/(TP+FN)
FPR = FP/(FP+TN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)

print(f'TPR = {TPR:.4f}, FPR = {FPR:.4f}, precision = {precision:.4f}, recall = {recall:.4f}')

TPR = 0.9000, FPR = 0.0092, precision = 0.0900, recall = 0.9000


***Исходя из roc_auc_curve:*** 

В первой модели FP меньше, чем во второй модели. Поэтому первая модель предпочтительнее. При этом FPR в первой модели и во второй модели различаются незначительно.

***Исходя из precision_recall_curve:***
Между первой моделью и второй моделью имеется существенная разница в значении precision. То есть precision_recall_curve позволяет легче сделать вывод в случае сильного дисбаланса классов.